In [ ]:
import pandas as pd
import os
import cv2
from typing import Tuple
import albumentations
import pandas as pd
import torch
import random
import time
import csv
import pickle
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T
from tqdm import tqdm
from torch import nn, optim
from torchvision import datasets, transforms, models
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.nn import functional as F
from PIL import Image
from torchvision.utils import make_grid
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix

In [ ]:
# SEED = 5566
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)

In [ ]:
class ResNetBase(nn.Module):
    def __init__(self, backbone='resnext50_32x4d_ssl'):
        super(ResNetBase, self).__init__()
        self.backbone = torch.hub.load(
            'facebookresearch/semi-supervised-ImageNet1K-models',
            backbone,
        )
        self.out_features = self.backbone.fc.in_features

    def forward(self, x):
        base = self.backbone
        x = base.conv1(x)
        x = base.bn1(x)
        x = base.relu(x)
        x = base.maxpool(x)

        x = base.layer1(x)
        x = base.layer2(x)
        x = base.layer3(x)
        x = base.layer4(x)
        return x

In [ ]:
class ResNetHead(nn.Module):
    def __init__(self, in_features: int, n_classes: int, use_neck: bool):
        super().__init__()

        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(in_features, n_classes)
        self.use_neck = use_neck

    def forward(self, x):
        if not self.use_neck:
            x = self.pooling(x)
            x = torch.flatten(x, start_dim=1)
        x = self.apply_fc_out(x)
        return x

    def apply_fc_out(self, x):
        return self.fc1(x)

In [ ]:
class Neck(nn.Module):
    def __init__(self, in_features: int, hidden_dim):
        super().__init__()

        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.bn1 = nn.BatchNorm1d(in_features)
        self.fc1 = nn.Linear(in_features, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)

    def forward(self, x):

        x = self.pooling(x)
        x = torch.flatten(x, start_dim=1)
        x = self.bn1(x)
        x = F.relu(self.fc1(x))
        x = self.bn2(x)
        x = F.relu(self.fc2(x))
        x = self.bn3(x)

        return x

In [ ]:
def build_model(backbone: str, n_classes: int, **kwargs) -> nn.Module:
    return Model(backbone=backbone, n_classes=n_classes, **kwargs)


class Model(nn.Module):
    def __init__(self, *, backbone: str, n_classes: int, use_neck: bool,):
        super().__init__()

        self.backbone = ResNetBase(backbone)
        self.in_features = self.backbone.out_features
        self.use_neck = use_neck
        if self.use_neck:
            self.hidden_dim = 1024
            self.neck = Neck(self.in_features, 1024)
            self.in_features = self.hidden_dim
        self.head = ResNetHead(self.in_features, n_classes, self.use_neck)

    def forward(self, x):
        x = self.backbone(x)
        if self.use_neck:
            x = self.neck(x)
        x = self.head(x)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
DATA_PATH = "/kaggle/input/shopee-code-league-20/_DA_Product_Detection"

TRAIN_CSV_PATH = os.path.join(DATA_PATH, "train.csv")
TRAIN_PATH = os.path.join(DATA_PATH, "train", "train")

train_df = pd.read_csv(TRAIN_CSV_PATH)

display(train_df)

In [ ]:
train_str_category = train_df.category.apply(lambda x: str(x).zfill(2))
train_df['file_path'] = train_str_category + os.sep + train_df.filename

display(train_df)
type(train_df)

In [ ]:
train_sample_path = os.path.join(TRAIN_PATH, train_df.iloc[1554].file_path)
Image.open(train_sample_path)

In [ ]:
type(train_df)

In [ ]:
plt.figure(figsize=(16,9))
plt.hist(train_df.category, bins=len(train_df.category.unique()),label=str)
plt.show()
print(train_df.category.value_counts().sort_index())

In [ ]:
class ProductImageDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.transform = transform
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_id, img_label = row['filename'], row['category']
        img_fname = self.root_dir + "/" + "{0:0=2d}".format(img_label) + "/" + str(img_id)
        img = Image.open(img_fname)
        img = img.convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_label

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_tfms = T.Compose([
    T.Resize((224,224)),
    T.RandomCrop(size=(224,224),padding=(10,10)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=15,fill=0),
     T.ToTensor(), 
     T.Normalize(*imagenet_stats), 
    #T.RandomErasing(inplace=True)
])

valid_tfms = T.Compose([
    T.Resize((224,224)), 
    T.ToTensor(), 
     T.Normalize(*imagenet_stats)
])

In [ ]:
train_set = ProductImageDataset(train_df, TRAIN_PATH, transform=train_tfms)
type(train_set)

In [ ]:
def show_sample(img, target):
    plt.imshow(img.permute(1, 2, 0))
    print('Labels:', target)
        
show_sample(*train_set[154])


In [ ]:
image, label = next(iter(train_set))
print("image", image)
print("Label", label)
print("Len", len(train_set))

In [ ]:
split_idx = int(train_df.shape[0] / 6 * 4)
split_idx

In [ ]:
(train_df.shape[0] - split_idx)//2

In [ ]:
split_idx = int(train_df.shape[0] / 6 * 4)
train_set, val_set, test_set = torch.utils.data.random_split(train_set,
            [split_idx, (train_df.shape[0] - split_idx)//2 + 1, (train_df.shape[0] - split_idx)//2],generator=torch.Generator().manual_seed(42))

In [ ]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, 
                      num_workers=3, pin_memory=True)

In [ ]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(16, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break
show_batch(train_loader)

In [ ]:
val_loader = DataLoader(
        val_set, batch_size=64,
        shuffle=False
)

In [ ]:
test_loader = DataLoader(
        test_set, batch_size=64,
        shuffle=False
)

In [ ]:
show_sample(*test_set[100])

In [ ]:
def plot_samples(df, data_root_path, category, num_imgs=20, num_cols=5):
    
    paths = df[df.category == category].file_path.sample(num_imgs)
    
    num_rows = num_imgs // num_cols 
    if num_imgs % num_cols != 0:
        num_rows += 1
    
    fig = plt.figure(figsize=(num_cols * 4, num_rows * 4))
    for idx, path in enumerate(paths):

        ax = fig.add_subplot(num_rows,num_cols,idx+1)
        
        im = cv2.imread(os.path.join(data_root_path, path))
        im_resized = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)

        plt.imshow(cv2.cvtColor(im_resized, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
        
    plt.show()
# for category in sorted(train_df.category.unique()):
#     print(category)
#     plot_samples(train_df, TRAIN_PATH, category, 5, 5)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
figure_dict ={
    "train_loss":[],
    "val_loss":[],
    "train_acc":[],
    "val_acc":[]
}


In [ ]:
train_losses = []
train_accu = []
def train_epoch(
        loader, model, device,
        criterion, optimizer, 
        n_classes,
):
    global train_accu, train_losses
    correct=0
    total=0
    running_loss=0
    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device, dtype=torch.float), target.to(device)
        
        optimizer.zero_grad()
        preds = model(data)
        loss = criterion(preds, target)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        _, predicted = preds.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        bar.set_description(f'loss: {loss_np:.5f}')
    accu=correct/total
    train_loss=running_loss/len(loader)
    train_accu.append(accu)
    train_losses.append(train_loss)  
    if epoch % 2 == 0:
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, "model_{}.pth".format(epoch))
    global firgure_dict
    figure_dict["train_loss"].append(train_loss)
    figure_dict["train_acc"].append(accu)
    return train_loss

In [ ]:
eval_losses=[]
eval_accu=[]
y_pred = []
y_true = []
def val_epoch(loader, model, device, criterion, n_classes):

    val_loss = 0
    acc = 0
    model.eval()
    
    with torch.no_grad():
        for (data, targets) in tqdm(loader):
            data, targets = data.to(device, dtype=torch.float), targets.to(device)
            
            preds = model(data)
            labels = targets.data.cpu().numpy()
            y_true.extend(labels)
            loss = criterion(preds, targets)
            
            val_loss += loss.item()
            acc += (preds.argmax(1) == targets).float().mean().item()
            preds = (torch.max(torch.exp(preds), 1)[1]).data.cpu().numpy()
            y_pred.extend(preds)
    n_total = len(loader)
    val_loss = val_loss / n_total
    acc = acc / n_total
    eval_losses.append(val_loss)
    eval_accu.append(acc)
    #top_acc = top_acc.compute()
    global firgure_dict
    figure_dict["val_loss"].append(val_loss)
    figure_dict["val_acc"].append(acc)
    print('acc', acc)
    return val_loss, acc

In [ ]:
model = build_model(
        backbone='resnext50_32x4d_ssl', n_classes=42,
        use_neck=1,
    )
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer.zero_grad()
optimizer.step()
criterion_fn = nn.CrossEntropyLoss()
n_classes=42

In [ ]:
checkpoint = torch.load('../input/123toan/model_20-30.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
for epoch in range(1, 5):
    print(time.ctime(), 'Epoch:', epoch)
    train_loss = train_epoch(
        train_loader, model,
        device, criterion_fn, optimizer,
        n_classes,
    )
    val_loss, acc = val_epoch(
        val_loader, model,
        device, criterion_fn,
        n_classes,
    )

In [ ]:
with open('filename34-44.pickle_new', 'wb') as handle:
    pickle.dump(figure_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../input/123toan/filename1-6.pickle_new', 'rb') as handle:
    dict1 = pickle.load(handle)

In [ ]:
with open('../input/123toan/filename6-20.pickle_new', 'rb') as handle:
    dict2 = pickle.load(handle)

In [ ]:
with open('../input/123toan/filename20-34.pickle_new', 'rb') as handle:
    dict3 = pickle.load(handle)

In [ ]:
with open('./filename34-44.pickle_new', 'rb') as handle:
    dict4 = pickle.load(handle)

In [ ]:
print(len(y_pred))
print(len(y_true))

In [ ]:
# classes = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']
# print(len(multilabel_confusion_matrix(y_true, y_pred)))
# x = multilabel_confusion_matrix(y_true, y_pred)
# df_cm = pd.DataFrame(x/np.sum(x) *10, index =2],
#                      columns = 2])
# plt.figure(figsize = (32,15))
# sn.heatmap(df_cm, annot=True)
# # plt.savefig('output.png')
# print(x)


In [ ]:
for key, value in figure_dict.items():
    print(key, ' : ', value)

In [ ]:
train_acc=dict1["train_acc"]+dict2["train_acc"]+dict3["train_acc"]+dict4["train_acc"]
val_acc=dict1['val_acc']+dict2['val_acc']+dict3['val_acc']+dict4['val_acc']
train_loss=dict1['train_loss']+dict2['train_loss']+dict3['train_loss']+dict4['train_loss']
val_los=dict1['val_loss']+dict2['val_loss']+dict3['val_loss']+dict4['val_loss']

In [ ]:
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')
 
plt.show()

In [ ]:
plt.plot(train_loss)
plt.plot(val_los)
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Losses')
 
plt.show()

In [ ]:
# cf_matrix = confusion_matrix(y_true, y_pred)
# temp_df = pd.DataFrame(confusion_matrix(y_true, y_pred) / np.sum(cf_matrix) * 10).applymap(lambda x : int(x * 100) / 100)

In [ ]:
classes = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']
report = classification_report(y_true, y_pred, target_names=classes,output_dict=True)
for key, value in report.items():
    print(key, ' : ', value)

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_tfms = T.Compose([
    T.Resize((224,224)),
    T.RandomCrop(size=(224,224),padding=(10,10)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=15,fill=0),
     T.ToTensor(), 
     T.Normalize(*imagenet_stats), 
    #T.RandomErasing(inplace=True)
])

valid_tfms = T.Compose([
    T.Resize((224,224)), 
    T.ToTensor(), 
     T.Normalize(*imagenet_stats)
])

In [ ]:
test_true = []
test_pred = []
def test_epoch(loader, model, device, n_classes):
    
    acc = 0
    model.eval()
    
    with torch.no_grad():
        for (data, targets) in tqdm(loader):
            data, targets = data.to(device, dtype=torch.float), targets.to(device)
            
            preds = model(data)
            labels = targets.data.cpu().numpy()
            test_true.extend(labels)
            acc += (preds.argmax(1) == targets).float().mean().item()
            preds = (torch.max(torch.exp(preds), 1)[1]).data.cpu().numpy()
            test_pred.extend(preds)
    n_total = len(loader)
    acc = acc / n_total
    print('acc', acc)
    return acc

In [ ]:
acc = test_epoch(
    test_loader, model,
    device,
    n_classes,
)

In [ ]:
classes = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']
report = classification_report(test_true, test_pred, target_names=classes,output_dict=True)
for key, value in report.items():
    print(key, ' : ', value)

In [ ]:
# test_dl = DataLoader(test_dataset, batch_size=64, shuffle=True, 
#                       num_workers=3, pin_memory=True)

In [ ]:
test_preds = []
test_trues = []
def tests_epoch(loader, model, device, n_classes):
    
    acc = 0

    with torch.no_grad():
        for (data, targets) in tqdm(loader):
            data, targets = data.to(device, dtype=torch.float), targets.to(device)
            
            preds = model(data)
            labels = targets.data.cpu().numpy()
            test_trues.extend(labels)
            
            acc += (preds.argmax(1) == targets).float().mean().item()
            preds = (torch.max(torch.exp(preds), 1)[1]).data.cpu().numpy()
            test_preds.extend(preds)
            
    n_total = len(loader)
    acc = acc / n_total
    print('acc',acc)
    return acc

In [ ]:
TEST_csv_path = "../input/123sdfbgdfbfd/test1.csv"
TEST_path = '../input/123sdfbgdfbfd/sorted/sorted'
test_Dataframe = pd.read_csv(TEST_csv_path)
display(test_Dataframe)

In [ ]:
new=test_Dataframe[test_Dataframe.category!=42]
display(new)

In [ ]:
new=new.reset_index()

In [ ]:
test_Set = ProductImageDataset(new, TEST_path, transform=valid_tfms)
test_dl = DataLoader(test_Set, batch_size=64, shuffle=True, 
                      num_workers=3, pin_memory=True)

In [ ]:
def transform(img):
    transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((512,512)),
            transforms.RandomCrop(size=(512,512),padding=(10,10)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
    ])
    return transform

In [ ]:
new_data = []
for img,label in test_dl:
    new_data.append(transform(img))
#     new_data.append(img)
#     new_data.append(img)
#     new_data.append(img)


In [ ]:
plt.figure()
plt.hist(new.category, bins=len(new.category.unique()))
plt.show()
print("Total test set: ",len(new))
print(new.category.value_counts().sort_index())

In [ ]:
acc = tests_epoch(
    test_dl, model,
    device,n_classes,
)

In [ ]:
report = classification_report(test_trues, test_preds, target_names=classes,output_dict=True)
for key, value in report.items():
    print(key, ' : ', value)

In [ ]:
cm = confusion_matrix(test_trues, test_preds)
print(cm)

plt.figure(figsize=(30,20))
cf = sn.heatmap(cm,annot=True)
plt.show(cf)